#**INTRODUCCIÓN AL PROCESAMIENTO DE LENGUAJE NATURAL**


Existen muchas formas de datos y una importante es el **TEXTO**.

Hay muchas librerías para NLP una de ellas es [spaCy](https://spacy.io/).

SpaCy trabaja sobre modelos específicos de los lenguaje, que deben leerse.

In [1]:
#Lenguaje español
!spacy download es_core_news_sm

     |████████████████████████████████| 16.2 MB 5.2 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172933 sha256=0b80c8788ec061d9e00f339b81e4818d5c3c4c301d4a5d3a4920411bba00a5f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-197ueflj/wheels/21/8d/a9/6c1a2809c55dd22cd9644ae503a52ba6206b04aa57ba83a3d8
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


Reiniciar el entorno de ejecución

In [1]:
import spacy 
nlp = spacy.load('es_core_news_sm')

Una vez cargado el modelo se puede procesar texto.

Un documento está formado por **tokens**, que pueden verse a través de una iteración.

In [2]:
texto = 'yo leo, tú lees, ella lee, nosotros leemos, ustedes leen.'
doc = nlp(texto)

for token in doc:
  print (token)

yo
leo
,
tú
lees
,
ella
lee
,
nosotros
leemos
,
ustedes
leen
.


Cada token contiene información adicional. Por ejemplo, token.lemma_ y token.is_stop.

Lemma es la forma base, ejemplo "caminar" es el lemma de "caminando".

Frecuentemente, es necesario eliminar palabras que no tienen mucha información, por ejemplo, "el", "ella", "no", 

In [3]:

lemmas = [tok.lemma_.lower() for tok in doc]
print(len(lemmas))
print (lemmas)

15
['yo', 'leer', ',', 'tú', 'leer', ',', 'él', 'leer', ',', 'nosotros', 'leer', ',', 'vosotros', 'leer', '.']


In [4]:
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma', 'Stopword'))
print("-"*50)
for token in doc:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}")

Token 		Lemma 		Stopword
--------------------------------------------------
yo		yo		True
leo		leer		False
,		,		False
tú		tú		True
lees		leer		False
,		,		False
ella		él		True
lee		leer		False
,		,		False
nosotros		nosotros		True
leemos		leer		False
,		,		False
ustedes		vosotros		True
leen		leer		False
.		.		False


Para que ponga qué es cada una de las palabras

In [5]:
doc1 = nlp ("Esto es una prueba de análisis de texto, para ver cómo funciona")
for p in doc1:
  print (p, p.pos_)

Esto PRON
es AUX
una DET
prueba NOUN
de ADP
análisis NOUN
de ADP
texto NOUN
, PUNCT
para ADP
ver AUX
cómo PRON
funciona VERB


Para analizar por sentencias

In [6]:
doc2 = nlp ("En torno de una mesa de cantina. una noche de invierno. "+
            "Regocijadamente departían seis alegres bohemios." + 
            "Los ecos de sus risas se escapaban. " + 
            "Y de aquel barrio quieto. " +  
            "Iban a interrumpir el impotente." +
            "y profundo silencio.")

for frase in doc2.sents:
  print (frase)      

En torno de una mesa de cantina.
una noche de invierno.
Regocijadamente departían seis alegres bohemios.
Los ecos de sus risas se escapaban.
Y de aquel barrio quieto.
Iban a interrumpir el impotente.y profundo silencio.


Por componentes

In [7]:
doc3 = nlp ("Un buen sistema educativo contribuye a reducir desigualdades y a lograr sociedades paritarias, tolerantes y pacíficas - En la actualidad, todavía hay más de 265 millones de niños y niñas que no están escolarizados (el 22% en edad primaria)")
for parte in doc3.noun_chunks:
  print (parte)

Un buen sistema
desigualdades
sociedades
tolerantes
la actualidad
millones
niños
niñas
que
edad


Mapeo de patrones

El mapeo de patrones puede realizarse a través de expresiones regulares, pero es más eficiente hacerlo creando un **`Matcher`** y usando **`PhraseMatcher`**.

El mapeo es una tupla de id y las posiciones iniciales y finales de la frase.

In [8]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher (nlp.vocab, attr = "LOWER") #mapeo de las frases en minúscula

terms = ['Galaxy', "iPhone 15", "Google Pixel"]
patrones = [nlp(text) for text in terms]
matcher.add("termsList", patrones)

In [24]:
patrones

[Galaxy, iPhone 15, Google Pixel]

In [9]:
doc4 = nlp ("Pruebas de fotografía enfrentan al iPhone 15 contra el Galaxy y el Google Pixel")
m = matcher (doc4)
print (m)

[(15896243148243938421, 5, 7), (15896243148243938421, 9, 10), (15896243148243938421, 12, 14)]


In [14]:
m_id, inicio, fin = m [0]
print (nlp.vocab.strings[m_id], doc4[inicio:fin])

termsList iPhone 15


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Ejercicio

Imagine que desea identificar cuándo un platillo de un menú no es agradable.

Considere el archivo `restaurant.json`, donde se puede determinar qué le gusta y qué no a las personas.

In [54]:
import pandas as pd
path = "/content/drive/MyDrive/CursoPythonAvanzado/BreveRepasoDePython/Datasets/restaurant.json"
data = pd.read_json (path)
data.head ()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [55]:
#considere que los platillos del menú son
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

**Paso 1:** Planear el análisis

¿Qué ideas tendría para encontrar que platillos del menú podrían mejorarse?

1. Agrupar las revisiones de acuerdo a los platillos que mencionan
2. Calcular el rating promedio de las revisiones de cada platillo
3. Determinar qué platillos tienen las calificaciones más bajas



**Paso 2:** Encontrar los items en una revisión

In [59]:
import spacy
from spacy.matcher import PhraseMatcher

index_of_review_to_test_on = 14   #podría ser cualquiera
text_to_test_on = data.text.iloc[index_of_review_to_test_on]
print ("Revisión: ", text_to_test_on)

# Cargar el modelo del idioma
nlp = spacy.blank('en')

# Determinar los tokens
review_doc = nlp(text_to_test_on)

# Crear un objeto PhraseMatcher. Usar attr = 'LOWER' 
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Crear una lista con los tokens de cada platillo del menu
menu_tokens_list = [nlp(item) for item in menu]

 
matcher.add("MENU",            
            menu_tokens_list 
           )

# Encontrar los mapeos en las revisiones (review_doc)
matches = matcher(review_doc)

for match in matches:
  print(f"Número de Token {match[1]}: {review_doc[match[1]:match[2]]}")
  #print(match)

Revisión:  The Il Purista sandwich has become a staple of my life. Mozzarella, basil, prosciutto, roasted red peppers and balsamic vinaigrette blend into a front runner for the best sandwich in the valley. Goes great with sparkling water or a beer. 

DeFalco's also has other Italian fare such as a delicious meatball sub and classic pastas.
Número de Token 2: Purista
Número de Token 16: prosciutto
Número de Token 58: meatball


**Paso 3:** Hacer el mapeo para todo el dataset

In [61]:
from collections import defaultdict

# item_ratings es un diccionario de listas. Si una llave no existe en item_ratings,
# se inserta con una lista vacía como valor
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
    # Mismo matcher del ejercicio anterior
    matches = matcher(doc)
    
    # Crea un conjunto de los items encontrados en el texto considerado
    found_items = set ([doc[match[1]:match[2]].text.lower() for match in matches])
    
    # Actualizar item_ratings con el rating de cada item en found_items
    # Transformar el item a minúsculas
    for item in found_items:
      item_ratings[item].append(review.stars)


**Paso 4:** ¿Cuál es el item con la peor revisión?

In [62]:
# Calcular el promedio de los ratings para cada item como un diccionario
mean_ratings = {item: sum (ratings)/len(ratings) for item, ratings in item_ratings.items ()}

# Encontrar el peor item, escribirlo
worst_item = sorted (mean_ratings, key=mean_ratings.get)[0]
print(worst_item)

#Escribir el rating promedio
print(mean_ratings[worst_item])

chicken cutlet
3.4


**Paso 5:** ¿Son relevantes los conteos ?

In [63]:
conteos = {item: len(ratings) for item, ratings in item_ratings.items()}

itemCont = sorted(conteos, key=conteos.get, reverse=True)
for item in itemCont:
    print(f"{item:>25}{conteos[item]:>5}")

                    pizza  265
                    pasta  206
                 meatball  128
              cheesesteak   97
             cheese steak   76
                  cannoli   72
                  calzone   72
                 eggplant   69
                  purista   63
                  lasagna   59
          italian sausage   53
               prosciutto   50
             chicken parm   50
             garlic bread   39
                  gnocchi   37
                spaghetti   36
                 calzones   35
                   pizzas   32
                   salami   28
            chicken pesto   27
             italian beef   25
                 tiramisu   21
            italian combo   21
                     ziti   21
         chicken parmesan   19
       chicken parmigiana   17
               portobello   14
           mac and cheese   11
           chicken cutlet   10
         steak and cheese    9
                 pastrami    9
               roast beef    7
       f

In [66]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Items peor evaluados:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Calificación promedio: {mean_ratings[item]:.2f} \tmenciones: {conteos[item]}")
    
print("\n\nItems mejor evaluados:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Calificación promedio: {mean_ratings[item]:.2f} \tmenciones: {conteos[item]}")

Items peor evaluados:
chicken cutlet       Calificación promedio: 3.40 	menciones: 10
turkey sandwich      Calificación promedio: 3.80 	menciones: 5
spaghetti            Calificación promedio: 3.89 	menciones: 36
italian beef         Calificación promedio: 3.92 	menciones: 25
tuna salad           Calificación promedio: 4.00 	menciones: 5
macaroni             Calificación promedio: 4.00 	menciones: 5
italian combo        Calificación promedio: 4.05 	menciones: 21
garlic bread         Calificación promedio: 4.13 	menciones: 39
roast beef           Calificación promedio: 4.14 	menciones: 7
eggplant             Calificación promedio: 4.16 	menciones: 69


Items mejor evaluados:
chicken pesto        Calificación promedio: 4.56 	menciones: 27
chicken salad        Calificación promedio: 4.60 	menciones: 5
purista              Calificación promedio: 4.67 	menciones: 63
prosciutto           Calificación promedio: 4.68 	menciones: 50
reuben               Calificación promedio: 4.75 	menciones: 4

Representación gráfica de la estructura

In [71]:
from spacy import displacy

doc4 = nlp ("Google compró Motorola por 12,500 millones de dólares el 11 agosto de 2011")
displacy.render (doc4, style='dep', jupyter=True) #análisis por dependencias


/usr/lib/python3.7/runpy.py:193: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  "__main__", mod_spec)


In [70]:
displacy.render (doc4, style='ent', jupyter=True) #análisis por entidades

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [73]:
nlpE = spacy.load ("en_core_web_sm")
doc5 = nlpE("Russian armies were mobilized because of ISIS attacks in Syria on 25th of May")
displacy.render(doc5, style='ent', jupyter=True)